In [80]:
import numpy as np

import math 

from ad import adnumber, jacobian

from ad.admath import * 

## Car Packing Problem 

Huber loss parameters:

In [32]:
p_x = 0.1
p_y = 0.1
p_delta = 0.01
p_v = 1

State and control loss parameters:

In [ ]:
c_w = 0.01
c_a = 0.0001

Euler schema function:

In [81]:
def F(x, y, delta, v, w, a) -> "R^4*":
    f = h*v
    b = f*math.cos(w) + d - math.sqrt(d**2 - f**2 * math.sin(w)**2)

    x = x + b*math.cos(delta)
    y = y + b*math.sin(delta)
    delta = delta + math.asin(math.sin(w) * f/d)
    v = v + h*a
    
    return np.array([x, y, delta, v])

Functions to compute differentials of loss:

In [49]:
def DL(x, y, delta, v, w, a) -> "R^6, R^6*6":
    """Compute the gradient vector and hessian matrix of loss function."""
    x, y, delta, v, w, a = adnumber([x, y, delta, v, w, a])
    
    z_x = sqrt(x**2 + p_x**2) - p_x
    z_y = sqrt(y**2 + p_y**2) - p_y
    z_delta = sqrt(delta**2 + p_delta**2) - p_delta
    z_v = sqrt(v**2 + p_v**2) - p_v
    
    L = 0.01*(z_x + z_y) + c_w * w**2 + c_a * a**2
    
    return np.array(L.gradient([x, y, delta, v, w, a])), np.array(L.hessian([x, y, delta, v, w, a]))

Functions to compute differentials of F (of "Euler schema"):

In [50]:
def DF(x, y, delta, v, w, a) -> "R^4*6, R^4*6*6":
    """Compute the jacobian matrix and hessian tensor of Euler step."""
    x, y, delta, v, w, a = adnumber([x, y, delta, v, w, a])
    
    f = h*v
    b = f*cos(w) + d - sqrt(d**2 - f**2 * sin(w)**2)

    F_x = x + b*cos(delta)
    F_y = y + b*sin(delta)
    F_delta = delta + asin(sin(w) * f/d)
    F_v = v + h*a
    
    jaco = np.array(jacobian([F_x, F_y, F_delta, F_v], [x, y, delta, v, w, a]))
    
    H_x = F_x.hessian([x, y, delta, v, w, a])
    H_y = F_y.hessian([x, y, delta, v, w, a])
    H_delta = F_delta.hessian([x, y, delta, v, w, a])
    H_v = F_v.hessian([x, y, delta, v, w, a])
    
    hess = np.array([H_x, H_y, H_delta, H_v])
    
    return jaco, hess

Optimization initial settings:

In [71]:
from scipy.signal import unit_impulse
basis = [unit_impulse(6,i) for i in range(6)]

T = 1; N = 100; h = T/N; 
p = np.zeros(4)
X = np.array(N * [[1, 1, 3/2*pi, 0]])
U = np.zeros((N,2))

d = 4

* Calculus of differentials of final loss:

In [75]:
x, y, delta, v = adnumber(X[-1])
w, a = adnumber(U[-1])

z_x = sqrt(x**2 + p_x**2) - p_x
z_y = sqrt(y**2 + p_y**2) - p_y
z_delta = sqrt(delta**2 + p_delta**2) - p_delta
z_v = sqrt(v**2 + p_v**2) - p_v

L_F = z_x + z_y + z_delta + z_v

DVstar_list_inv = [np.array(L_F.gradient([x, y, delta, v]))] #DV*_{n-1}(x_{n-1})
D2Vstar_list_inv = [np.array(L_F.hessian([x, y, delta, v]))]

DV_list_inv = []
D2V_list_inv = []

* Backward pass

In [77]:
#backward pass, begin with DV_n-2
for t in range(N-2, -1, -1): #from N-2 to 0
    
    gradient_L, hessian_L = DL(*X[t], *U[t])
    jacobian_F, hessian_F = DF(*X[t], *U[t])
    DV = gradient_L + DVstar_list_inv[-1] @ jacobian_F
    D2V = np.reshape([ei @ hessian_L @ ej + 
                      DVstar_list_inv[-1] @ (ej @ hessian_F @ ei) + 
                      (jacobian_F @ ej) @ D2Vstar_list_inv[-1] @ (jacobian_F @ ei) for ei in basis for ej in basis], (6,6))

    DV_list_inv.append(DV)
    D2V_list_inv.append(D2V)
    
    DVstar = DV[:4] + DV[4:] @ np.linalg.inv(D2V[4:, 4:]) @ D2V[4:, :4]
    D2Vstar = D2V[:4, :4] + D2V[:4, 4:] @ np.linalg.inv(D2V[4:, 4:]) @ D2V[4:, :4]
   
    DVstar_list_inv.append(DVstar)
    D2Vstar_list_inv.append(D2Vstar)

* Forward pass

In [82]:
DV = DV_list_inv[::-1]
D2V = D2V_list_inv[::-1]

X_hat = np.copy(X)
#forward pass
for t in range(N-1):
    if t == 0:
        h_u = -np.linalg.inv(D2V[t][4:, 4:]) @ DV[t][4:]
        U[t] = U[t] + h_u
        X_hat[t+1] = F(*X_hat[t], *U[t])
    else:
        h_x = X_hat[t] - X[t]
        h_u = -np.linalg.inv(D2V[t][4:, 4:]) @ (DV[t][4:] + D2V[t][4:, :4] @ h_x)
        U[t] = U[t] + h_u
        X_hat[t+1] = F(*X_hat[t], *U[t])

In [84]:
X_hat

array([[1.00000000e+00, 1.00000000e+00, 4.71238898e+00, 0.00000000e+00],
       [1.00000000e+00, 1.00000000e+00, 4.71238898e+00, 7.74563007e-09],
       [1.00000000e+00, 1.00000000e+00, 4.71238898e+00, 9.75887973e-09],
       [1.00000000e+00, 1.00000000e+00, 4.71238898e+00, 1.22954138e-08],
       [1.00000000e+00, 1.00000000e+00, 4.71238898e+00, 1.54912454e-08],
       [1.00000000e+00, 1.00000000e+00, 4.71238898e+00, 1.95177421e-08],
       [1.00000000e+00, 1.00000000e+00, 4.71238898e+00, 2.45908158e-08],
       [1.00000000e+00, 1.00000000e+00, 4.71238898e+00, 3.09825006e-08],
       [1.00000000e+00, 1.00000000e+00, 4.71238898e+00, 3.90355400e-08],
       [1.00000000e+00, 1.00000000e+00, 4.71238898e+00, 4.91817606e-08],
       [1.00000000e+00, 1.00000000e+00, 4.71238898e+00, 6.19652206e-08],
       [1.00000000e+00, 1.00000000e+00, 4.71238898e+00, 7.80713685e-08],
       [1.00000000e+00, 1.00000000e+00, 4.71238898e+00, 9.83637811e-08],
       [1.00000000e+00, 1.00000000e+00, 4.71238898e

In [85]:
U

array([[ 0.00000000e+00,  7.74563007e-07],
       [ 9.93396359e+09,  2.01324966e-07],
       [ 7.88458994e+09,  2.53653403e-07],
       [ 6.25800220e+09,  3.19583163e-07],
       [ 4.96698004e+09,  4.02649674e-07],
       [ 3.94229633e+09,  5.07307364e-07],
       [ 3.12900515e+09,  6.39168489e-07],
       [ 2.48349571e+09,  8.05303930e-07],
       [ 1.97115411e+09,  1.01462207e-06],
       [ 1.56450738e+09,  1.27834599e-06],
       [ 1.24175043e+09,  1.61061479e-06],
       [ 9.85576879e+08,  2.02924126e-06],
       [ 7.82250895e+08,  2.55666723e-06],
       [ 6.20870552e+08,  3.22116854e-06],
       [ 4.92783061e+08,  4.05837493e-06],
       [ 3.91120637e+08,  5.11318731e-06],
       [ 3.10432139e+08,  6.44219432e-06],
       [ 2.46390743e+08,  8.11671542e-06],
       [ 1.95561983e+08,  1.02266272e-05],
       [ 1.55219711e+08,  1.28851664e-05],
       [ 1.23200069e+08,  1.62349542e-05],
       [ 9.77856096e+07,  2.04555493e-05],
       [ 7.76133169e+07,  2.57729346e-05],
       [ 6.

In [12]:
# basis[0] @ np.array([np.eye(6), np.eye(6), np.eye(6)]) @ basis[0]
# A = np.random.rand(4, 6, 6)
# b = np.random.rand(6)
# c = np.random.rand(6)

# c@ A @ b == np.array([c @a @b for a in A])

# c@ A @ b

# np.array([c @a @b for a in A])

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'acos',
 'acosh',
 'acot',
 'acoth',
 'acsc',
 'acsch',
 'admath',
 'asec',
 'asech',
 'asin',
 'asinh',
 'atan',
 'atan2',
 'atanh',
 'ceil',
 'cos',
 'cosh',
 'cot',
 'coth',
 'csc',
 'csch',
 'degrees',
 'e',
 'erf',
 'erfc',
 'exp',
 'expm1',
 'fabs',
 'factorial',
 'floor',
 'gamma',
 'hypot',
 'isinf',
 'isnan',
 'lgamma',
 'ln',
 'log',
 'log10',
 'log1p',
 'phase',
 'pi',
 'polar',
 'pow',
 'radians',
 'rect',
 'sec',
 'sech',
 'sin',
 'sinh',
 'sqrt',
 'tan',
 'tanh',
 'trunc']